In [ ]:
#
# Valuation of European call options in Black-Scholes-Merton model
# 

# Analytical Black-Scholes-Merton(BSM) Formula

In [ ]:
%%latex
\begin{align}
C(S_{t}, K_{t},T,r,\sigma) = S_{t} \cdot N(d_{1}) -e^{-r(T-t)} \cdot K \cdot N(d_{2}) \\
N(d) = \frac{1}{\sqrt{2\pi}}\int_{-∞}^{d} e^{-\frac{1}{2}x^{2}}dx \\
d_{1} = \frac{\log\frac{S_{t}}{K}+(r+\frac{\sigma^{2}}{2})(T-t)}{\sigma\sqrt{T-t}} \\
d_{2} = \frac{\log\frac{S_{t}}{K}+(r-\frac{\sigma^{2}}{2})(T-t)}{\sigma\sqrt{T-t}}
\end{align}

In [ ]:
import math
import scipy.stats

In [ ]:
def bsm_call_value(S0, K, T, R, sigma):
    """Valuation of European call option in BSM model Analytical formula
        
    Parameters
    - S0 : float(initial stock/index level)
    - K : float(strike price)
    - T : float(maturity date : in year fractions)
    - r : float(constant risk-free short rate)
    - sigma : float(volatility factor in diffusion term)
    
    Returns
    - value : float(present value of the European call option)
    """
    
    SO = float(S0)
    d1 = ( (math.log(S0) / K) + (r + 0.5 * sigma ** 2) * T ) / (sigma * math.sqrt(T) )
    d2 = ( (math.log(S0) / K) + (r - 0.5 * sigma ** 2) * T ) / (sigma * math.sqrt(T) )
    value = ( S0 * scipy.stats.norm.cdf(d1, 0.0, 1.0) - K * math.exp(-r * T) * scipy.stats.norm.cdf(d2, 0.0, 1.0))
    # scipy.stats.norm.cdf : cumulative distribution function for normal distribution
    return value

In [ ]:
def bsm_vega(S0, K, T, r, sigma):
    """Vega of European option in BSM model
    
    Parameters
    - S0 : float(initial stock/index level)
    - K : float(strike price)
    - T : float(maturity date : in year fractions)
    - r : float(constant risk-free short rate)
    - sigma : float(volatility factor in diffusion term)
    
    Returns
    - vega : float(partial derivative of BSM formula with respect to sigma, i.e. Vega)
    
    """
    
    SO = float(S0)
    d1 = ( (math.log(S0) / K) + (r + 0.5 * sigma ** 2) * T ) / (sigma * math.sqrt(T) )
    vega = S0 * scipy.stats.norm.cdf(d1, 0.0, 1.0) * math.sqrt(T)
    
    return vega   
    

In [ ]:
def bsm_call_imp_vol(S0, K, T, r, C0, sigma_est, it=100):
    """Implied volatility of European call option in BSM model
    
    Parameters
    - SO : float(initial stock/intex level)
    - K : float(strike price)
    - T : float(maturity date : in year fractions)
    - r : float(constant risk-free short rate)
    - sigma_est : float(estimate of impl. volatility)
    - it : integer(number of iterations)
    
    Returns
    - sigma_est : float(numberically estimated implied volatility)
    """
    
    for index in range(it):
        sigma_est -= ((bsm_call_value(S0, K, T, r, sigma_est) - C0) / bsm_vega(S0, K, T, r, sigma_est))
        
    return sigma_est
    
    

In [ ]:
V0 = 17.6639

In [ ]:
r = 0.01

In [ ]:
import pandas as pd
h5 = pd.HDFStore('./vstoxx_data_31032014.h5', 'r')
futures_data = h5['futures_data']
options_data = h5['options_data']
h5.close()

In [ ]:
options_data['DATE'] = pd.to_datetime(options_data.DATE)
futures_data['DATE'] = pd.to_datetime(futures_data.DATE)
options_data['MATURITY'] = pd.to_datetime(options_data.MATURITY)
futures_data['MATURITY'] = pd.to_datetime(futures_data.MATURITY)

In [ ]:
options_data.info()

In [ ]:
options_data[['DATE', 'MATURITY', 'TTM', 'STRIKE', 'PRICE']].head()

In [ ]:
options_data['IMP_VOL'] = 0.0

In [ ]:
to1 = 0.5

for option in options_data.index:
    forward = futures_data[futures_data['MATURITY'] == \
                           options_data.loc[option]['MATURITY']]['PRICE'].values[0]
    if forward * (1-to1) < options_data.loc[option]['STRIKE'] < forward * (1 + to1):
        imp_vol = bsm_call_imp_vol(
            V0,
            options_data.loc[option]['STRIKE'],
            options_data.loc[option]['TTM'],
            r,
            options_data.loc[option]['PRICE'],
            sigma_est = 2.,
            it = 100
        )
        options_data['IMP_VOL'].loc[option] = imp_vol
                           

In [ ]:
futures_data['MATURITY']

In [ ]:
options_data.loc[46170]

In [ ]:
options_data.loc[46170]['STRIKE']

In [ ]:
plot_data = options_data[options_data['IMP_VOL'] > 0]

In [ ]:
maturities = sorted(set(options_data['MATURITY']))
maturities

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,8))
for maturity in maturities:
    data = plot_data[options_data.MATURITY == maturity]
    plt.plot(data['STRIKE'], data['IMP_VOL'],
            label=maturity.date(), lw=1.5)
    
    plt.plot(data['STRIKE'], data['IMP_VOL'], 'r.')
    
plt.grid(True)
plt.xlabel('strike')
plt.ylabel('implied volatility of volatility')
plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
plt.show()